# Коллаборативная фильтрация с учетом любимых групп

In [1]:
import vk                                                                       
import time                                                                     
import pickle                                                                   
import numpy as np 
import collections
import itertools
import webbrowser
from time import sleep
from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
from collections import defaultdict                                             
                                                                                
def save_obj(obj, name ):                                                       
    with open(name + '.pkl', 'wb') as f:                                        
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)                            
                                                                                
def load_obj(name ):                                                            
    with open(name + '.pkl', 'rb') as f:                                        
        return pickle.load(f)

In [ ]:
all_members = {}
all_members.update(load_obj('./Anya_persons'))
print(len(all_members))
all_members.update(load_obj('./Dasha_persons'))
print(len(all_members))
all_members.update(load_obj('./Maks_persond_processed'))
print(len(all_members))

In [3]:
groups_l = set()
for pers in all_members:
    groups_l.update(all_members[pers]['groups']) 
groups_l = list(groups_l)

In [4]:
groups = {idx:group for idx, group in enumerate(groups_l)}
reverse_groups = {group:idx for idx, group in enumerate(groups_l)}

In [5]:
print(max(groups.keys()))
print(max(reverse_groups.keys()))

1141951
189802470


In [6]:
users_l = list(all_members.keys())
users = {idx:user for idx, user in enumerate(users_l)}
reverse_users = {user:idx for idx, user in enumerate(users_l)}

In [27]:
best_groups = True

In [28]:

my_friends_for_test=[995864,6173127,6811139,8431101,9420850,10804323,11030995,14245770,17468315,22840792,25567189,25883865,29145732,49363671,62369794,66382695,78330794,171038940,178159310,182861903,292309514,38628143]
users_for_test =[995864,6173127,6811139,8431101,9420850,10804323,11030995,14245770,17468315,22840792,25567189,25883865,29145732,49363671,62369794,66382695,78330794,171038940,178159310,182861903,292309514,38628143,59401294,33436943,12653364,300102978,284227818,120889121,120522331,310242262,82046724,120942983,243561432,267294790,5179470,9558686,17260358,32670543,89754899,484844105,187961856,139106107,182453074,11311268,19370392,67495362]

In [29]:
data = []
row = []
col =[]
#boundaries
#подбираем границы по баллам для любимых групп
first = 0.03
second = 0.1
if best_groups:
    for idx, (user, val) in enumerate(all_members.items()):
        for group in val['groups'][0:int(first*len(val['groups']))]:
            row.append(idx)
            data.append(100)
            col.append(reverse_groups[group])
        for group in val['groups'][int(first*len(val['groups'])):int(second*len(val['groups']))]:
            row.append(idx)
            data.append(10)
            col.append(reverse_groups[group])
        for group in val['groups'][int(second*len(val['groups'])):]:
            row.append(idx)
            data.append(1)
            col.append(reverse_groups[group])
else:
    for idx, (user, val) in enumerate(all_members.items()):
        for group in val['groups']:
            row.append(idx)
            data.append(1)
            col.append(reverse_groups[group])
    

In [30]:
csr = csr_matrix((np.array(data), (np.array(row),np.array(col))))

In [31]:
csr.shape

(46170, 1141952)

In [32]:
def recs(idx, csr_a):
    # считаем косинус между всеми пользователями
    metrics = cosine_similarity(csr_a.getrow(idx).toarray().reshape(1,-1), csr_a).reshape(-1, 1)
    metrics[idx] = 0
    # домножаем оценки пользовтеля на коэффициент похожести

    csr_a = csr_a.multiply(metrics)
    total_rate = np.sum(csr_a, axis=0)
    return total_rate

In [33]:
def get_sort(csr_a, user_l):
    user = reverse_users[user_l]
    t = recs(user, csr) 
    a = t.tolist()
    a = a[0]
    a = [(j,i) for i , j in enumerate(a)]
    a.sort(key=lambda i: i[0], reverse=True)
    return a

In [34]:
def precision_for_one(csr, user_l):
    recomend = set([groups[idx] for score, idx in get_sort(csr, user_l)[:6]])
    groups_of_user = set(all_members[user_l]['groups'])
    TP = len(recomend&groups_of_user)
    return TP/6.

In [15]:
def open_links(is_groups, user_groups):
    user, groups = user_groups
    webbrowser.open('https://vk.com/id'+str(user))
    for group in groups:
        sleep(0.8)
        webbrowser.open('https://vk.com/public' + str(group))

In [59]:
import random
users_for_test = random.sample(users_l, 1000)

In [35]:
list_of_prec = []
out_users = []
for i in users_for_test:
    try:
        list_of_prec.append(precision_for_one(csr, i))
        out_users.append(i)
    except :
        continue
sum(list_of_prec)/len(list_of_prec)
# средний precision для рандомной выборки из 1000 пользователей:

0.6851851851851851

In [37]:
out = {}
drop = True
users_for_test = [9183085,30811126]
for user_id in users_for_test:
    try:
        gr = get_sort(csr, user_id)
        out[user_id] = {}
        out[user_id]['groups'] = []
        out[user_id]['method'] = 1
        for g in gr:
            if drop:
                if groups[g[1]] not in all_members[user_id]['groups']:
                    out[user_id]['groups'].append(groups[g[1]])
            else:
                out[user_id].append(groups[g[1]])
            if len(out[user_id]['groups']) == 6:
                break
    except :
        continue


In [112]:
open_links(True, list(out.items())[2])

In [38]:
list(out.items())[1]

(30811126,
 {'groups': [43776215, 57846937, 91050183, 41437811, 139923997, 29559271],
  'method': 1})

In [39]:
save_obj(out,'Maks_recs_Dasha_Anya_1')

In [139]:
save_obj(out,'Maks_recs_0.pkl')

In [140]:
len(out)

36

In [13]:
maks = reverse_users[57583179]

In [14]:
t = recs(maks, csr)

In [16]:
a = t.tolist()
a = a[0]
a = [(j,i) for i , j in enumerate(a) ]
#скор номер
a.sort(key=lambda i: i[0], reverse=True)
i = [(s,i) for s, i in a if groups[i] not in all_members[57583179]['groups']][:10]
print(['https://vk.com/public'+str(groups[idx]) for score, idx in i])

['https://vk.com/public65652356', 'https://vk.com/public29534144', 'https://vk.com/public78019879', 'https://vk.com/public38664418', 'https://vk.com/public63731512', 'https://vk.com/public30022666', 'https://vk.com/public76982440', 'https://vk.com/public29559271', 'https://vk.com/public24098496', 'https://vk.com/public34215577']


In [59]:
import webbrowser

# generate an URL
url = 'https://vk.com/write25883865'
webbrowser.open(url)

True